# Using Pyserini BM25 implementation to search queries on TREC-DL 2020

## Setup

In [ ]:
!mkdir collections

In [3]:
!mkdir collections/msmarco-passage

In [4]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage

--2023-03-04 16:35:13--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  5.98MB/s    in 1m 55s  

2023-03-04 16:37:09 (8.78 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]



In [5]:
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


In [4]:
!pip install -q condacolab

In [5]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [10]:
!conda install -c conda-forge maven

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - maven


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.8             |       h166bdaf_0         578 KB  conda-forge
    cairo-1.16.0               |    ha61ee94_1014         1.5 MB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    colorama-0.4.6             |     pyhd8ed1ab_0          25 KB  conda-forge
    conda-23.1.0               |   py38h578d9bd_0         907 KB  conda-forge
    expat-2.5.0                |       h27087fc_0         189 KB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0          94

In [12]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules

Cloning into 'pyserini'...
remote: Enumerating objects: 7217, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 7217 (delta 155), reused 195 (delta 115), pack-reused 6954
Receiving objects: 100% (7217/7217), 4.25 MiB | 22.22 MiB/s, done.
Resolving deltas: 100% (5201/5201), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 23.83 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '7b84f773225b5973b4533dfa0aa18653409a6146'


In [14]:
!cd pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

In [16]:
!pip install -e pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/pyserini
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [17]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 43.7 MB/s eta 0:00:00


In [18]:
!conda install faiss-cpu -c pytorch

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - faiss-cpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    faiss-1.7.2                |py38h955d4c5_3_cpu         1.0 MB  conda-forge
    faiss-cpu-1.7.2            |       h718b53a_3          15 KB  conda-forge
    libblas-3.9.0              |16_linux64_openblas          13 KB  conda-forge
    libcblas-3.9.0             |16_linux64_openblas          13 KB  conda-forge
    libfaiss-1.7.2             |   h5ceb0a5_3_cpu         941 KB  conda-forge
    libfaiss-avx2-1.7.2        |   

In [19]:
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [22]:
!python pyserini/tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path collections/msmarco-passage/collection.tsv \
 --output-folder collections/msmarco-passage/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

In [24]:
!git clone https://github.com/castorini/anserini.git --recurse-submodules

Cloning into 'anserini'...
remote: Enumerating objects: 25491, done.
remote: Counting objects: 100% (1087/1087), done.
remote: Compressing objects: 100% (616/616), done.
remote: Total 25491 (delta 647), reused 790 (delta 411), pack-reused 24404
Receiving objects: 100% (25491/25491), 73.61 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (16379/16379), done.
Updating files: 100% (2260/2260), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/anserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 18.41 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '95fbaf2af75e2b59304ac5702d5479d50f3bd9ef'


In [28]:
!mvn -f anserini/pom.xml clean package appassembler:assemble

[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.pom (12 kB at 15 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/33/maven-plugins-33.pom (11 kB at 198 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/33/maven-parent-33.pom (44 kB at 519 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/21/apache-21.pom (17 kB at 900 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.jar (62 kB at 1.7 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/mojo/appassembler-maven-plugin/2.1.0/appassembler-maven-plugin-2.1.0.pom (19 kB at 885 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/code

In [30]:
!ls anserini/target/

anserini-0.20.1-SNAPSHOT-fatjar.jar   idx-sample-fw1677949900758
anserini-0.20.1-SNAPSHOT.jar	      idx-sample-fw-stored1677949900230
anserini-0.20.1-SNAPSHOT-javadoc.jar  idx-sample-ll
anserini-0.20.1-SNAPSHOT-sources.jar  idx-sample-ll1677949900108
apidocs				      idx-sample-ll1677949900174
appassembler			      idx-sample-ll1677949900637
ast1677949901438		      idx-sample-ll1677949900719
ast1677949901487		      jacoco.exec
ast1677949901531		      javadoc-bundle-options
ast1677949901580		      maven-archiver
classes				      maven-javadoc-plugin-stale-data.txt
generated-sources		      maven-status
generated-test-sources		      site
idx-sample-fw			      surefire-reports
idx-sample-fw1677949899854	      test-classes
idx-sample-fw1677949900680


In [32]:
!cp anserini/target/anserini-0.20.1-SNAPSHOT-fatjar.jar pyserini/pyserini/resources/jars

## Experiments

In [33]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/msmarco-passage/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-03-04 17:21:09,787 INFO  [main] index.IndexCollection (IndexCollection.java:393) - Setting log level to INFO
2023-03-04 17:21:09,790 INFO  [main] index.IndexCollection (IndexCollection.java:396) - Starting indexer...
2023-03-04 17:21:09,790 INFO  [main] index.IndexCollection (IndexCollection.java:397) - ============ Loading Parameters ============
2023-03-04 17:21:09,790 INFO  [main] index.IndexCollection (IndexCollection.java:398) - DocumentCollection path: collections/msmarco-passage/collection_jsonl
2023-03-04 17:21:09,791 INFO  [main] index.IndexCollection (IndexCollection.java:399) - CollectionClass: JsonCollection
2023-03-04 17:21:09,791 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Generator: DefaultLuceneDocumentGenerator
2023-03-04 17:21:09,792 INFO  [main] index.IndexCollection (IndexCollection.java:401) - Threads: 9
2023-03-04 17:21:09,792 INFO  [main] index.IndexCollection (IndexCollection.java:402) - Language: en
2023-03-04 17:21:09,792 INFO  [main] 

OBS.: Thanks to [Marcus Borela](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula1_buscador_simples.ipynb) and [Gustavo Guedes](https://colab.research.google.com/drive/10z86PObSxqbXczZ9pz0T-QurL21oMShf?usp=sharing) tips on the correct eval dataset to be used.

In [44]:
topics = 'dl20'
qrels_eval = 'dl20-passage'

In [52]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {topics} \
  --output runs/run.{topics}.bm25tuned.txt \
  --output-format msmarco \
  --hits 100 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running dl20 topics, saving to runs/run.dl20.bm25tuned.txt...
100% 200/200 [00:13<00:00, 14.91it/s]


In [53]:
!python pyserini/tools/scripts/msmarco/msmarco_passage_eval.py \
   pyserini/tools/topics-and-qrels/qrels.{qrels_eval}.txt runs/run.{topics}.bm25tuned.txt

#####################
MRR @10: 1.0
QueriesRanked: 200
#####################


In [54]:
!python -m pyserini.eval.convert_msmarco_run_to_trec_run \
   --input runs/run.{topics}.bm25tuned.txt \
   --output runs/run.{topics}.bm25tuned.trec

Done!


In [57]:
!python pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input pyserini/tools/topics-and-qrels/qrels.{qrels_eval}.txt \
   --output collections/msmarco-passage/qrels.{qrels_eval}.trec

Done!


In [60]:
!pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   collections/msmarco-passage/qrels.{qrels_eval}.trec runs/run.{topics}.bm25tuned.trec

map                   	all	0.2695
ndcg_cut_10           	all	0.4876
